In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r '/content/drive/My Drive/Colab Notebooks/[Kaggle] Understanding Clouds from Satellite Images/Scripts/.' .

In [0]:
!unzip -q '/content/drive/My Drive/Colab Notebooks/[Kaggle] Understanding Clouds from Satellite Images/Data/train_images320x480.zip'

### Dependencies

In [4]:
from utillity_script_cloud_segmentation import *

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [5]:
seed = 0
seed_everything(seed)
warnings.filterwarnings("ignore")

In [0]:
base_path = '/content/drive/My Drive/Colab Notebooks/[Kaggle] Understanding Clouds from Satellite Images/'
data_path = base_path + 'Data/'
model_base_path = base_path + 'Models/files/classification/'
model_path = model_base_path + '26-EfficientNetB0_224x224.h5'
hold_out_set_path = data_path + 'hold-out.csv'
train_images_dest_path = 'train_images/'

### Load data

In [7]:
hold_out_set = pd.read_csv(hold_out_set_path)
X_val = hold_out_set[hold_out_set['set'] == 'validation']
print('Validation samples: ', len(X_val)) 

# Preprocecss data
label_columns=['Fish', 'Flower', 'Gravel', 'Sugar']
for label in label_columns:
  X_val[label].replace({0: 1, 1: 0}, inplace=True)

display(X_val.head())

Validation samples:  1105


,image,Fish_mask,Flower_mask,Gravel_mask,Sugar_mask,Fish,Flower,Gravel,Sugar,set
4420,3c61c1f.jpg,NaN,1250814 786 1252214 786 1253614 786 1255014 78...,424922 678 426322 678 427722 678 429122 678 43...,NaN,1,0,0,1,validation
4421,f894de0.jpg,NaN,NaN,NaN,732816 335 734216 335 735616 335 737016 335 73...,1,1,1,0,validation
4422,a68b8fb.jpg,NaN,1910606 289 1910897 18 1910920 1 1912006 291 1...,50604 1193 52004 1193 53404 1193 54804 1193 56...,24498 700 25898 700 27298 700 28698 700 30098 ...,1,0,0,0,validation
4423,25496aa.jpg,524010 541 525410 541 526810 541 528210 541 52...,49045 1200 50445 1200 51845 1200 53245 1200 54...,NaN,1145219 542 1146619 542 1148019 542 1149419 54...,0,0,1,0,validation
4424,7c7f158.jpg,12634 522 14034 522 15434 522 16834 522 18234 ...,1469169 476 1470569 476 1471969 476 1473369 47...,NaN,NaN,0,0,1,1,validation


# Model parameters

In [0]:
HEIGHT = 224
WIDTH = 224
BETA1 = 0.25
BETA2 = 0.5
BETA3 = 1
TTA_STEPS = 8

# Model

In [9]:
model = load_model(model_path)












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




### Classification data generator

In [10]:
datagen=ImageDataGenerator(rescale=1./255.,
                           vertical_flip=True,
                           horizontal_flip=True,
                           zoom_range=[1, 1.1],
                           shear_range=45.0,
                           rotation_range=360,
                           width_shift_range=0.1,
                           height_shift_range=0.1,
                           brightness_range=(0.9, 1),
                           fill_mode='constant',
                           cval=0.)

valid_generator=datagen.flow_from_dataframe(
                            dataframe=X_val,
                            directory=train_images_dest_path,
                            x_col="image",
                            y_col=label_columns,
                            target_size=(HEIGHT, WIDTH),
                            batch_size=1,
                            class_mode="other",
                            shuffle=False,
                            seed=seed)

Found 1105 validated image filenames.


# Classification threshold and mask size tunning

In [11]:
valid_preds = apply_tta(model, valid_generator, steps=TTA_STEPS)
print('BETA1')
best_tresholds1 = classification_tunning(valid_generator.labels, valid_preds, label_columns, beta=BETA1)
print('BETA2')
best_tresholds2 = classification_tunning(valid_generator.labels, valid_preds, label_columns, beta=BETA2)
print('BETA3')
best_tresholds3 = classification_tunning(valid_generator.labels, valid_preds, label_columns, beta=BETA3)

BETA1
Fish treshold=0.77 Score=0.775
Flower treshold=0.81 Score=0.930
Gravel treshold=0.67 Score=0.730
Sugar treshold=0.75 Score=0.757
BETA2
Fish treshold=0.60 Score=0.732
Flower treshold=0.68 Score=0.884
Gravel treshold=0.57 Score=0.701
Sugar treshold=0.62 Score=0.680
BETA3
Fish treshold=0.26 Score=0.745
Flower treshold=0.48 Score=0.864
Gravel treshold=0.42 Score=0.738
Sugar treshold=0.47 Score=0.668


# Metrics (beta 0.25)

In [12]:
get_metrics_classification(X_val, valid_preds, label_columns, best_tresholds1)

Metrics for: Fish
              precision    recall  f1-score   support

           0       0.57      0.97      0.72       555
           1       0.88      0.26      0.40       550

    accuracy                           0.62      1105
   macro avg       0.73      0.61      0.56      1105
weighted avg       0.73      0.62      0.56      1105

Metrics for: Flower
              precision    recall  f1-score   support

           0       0.66      0.96      0.78       473
           1       0.96      0.63      0.76       632

    accuracy                           0.77      1105
   macro avg       0.81      0.80      0.77      1105
weighted avg       0.83      0.77      0.77      1105

Metrics for: Gravel
              precision    recall  f1-score   support

           0       0.66      0.86      0.74       587
           1       0.75      0.49      0.60       518

    accuracy                           0.69      1105
   macro avg       0.70      0.67      0.67      1105
weighted avg    

# Metrics (beta 0.5)

In [13]:
get_metrics_classification(X_val, valid_preds, label_columns, best_tresholds2)

Metrics for: Fish
              precision    recall  f1-score   support

           0       0.67      0.84      0.75       555
           1       0.78      0.59      0.67       550

    accuracy                           0.71      1105
   macro avg       0.73      0.71      0.71      1105
weighted avg       0.73      0.71      0.71      1105

Metrics for: Flower
              precision    recall  f1-score   support

           0       0.74      0.91      0.82       473
           1       0.92      0.76      0.83       632

    accuracy                           0.83      1105
   macro avg       0.83      0.84      0.83      1105
weighted avg       0.84      0.83      0.83      1105

Metrics for: Gravel
              precision    recall  f1-score   support

           0       0.74      0.73      0.74       587
           1       0.70      0.71      0.70       518

    accuracy                           0.72      1105
   macro avg       0.72      0.72      0.72      1105
weighted avg    

# Metrics (beta 1)

In [14]:
get_metrics_classification(X_val, valid_preds, label_columns, best_tresholds3)

Metrics for: Fish
              precision    recall  f1-score   support

           0       0.86      0.44      0.58       555
           1       0.62      0.93      0.75       550

    accuracy                           0.68      1105
   macro avg       0.74      0.68      0.66      1105
weighted avg       0.74      0.68      0.66      1105

Metrics for: Flower
              precision    recall  f1-score   support

           0       0.83      0.79      0.81       473
           1       0.85      0.88      0.86       632

    accuracy                           0.84      1105
   macro avg       0.84      0.84      0.84      1105
weighted avg       0.84      0.84      0.84      1105

Metrics for: Gravel
              precision    recall  f1-score   support

           0       0.86      0.51      0.64       587
           1       0.62      0.91      0.74       518

    accuracy                           0.70      1105
   macro avg       0.74      0.71      0.69      1105
weighted avg    